# Reviews combination

This notebook allows you to combine all the reviews present in the various useful datasets identified to create a single dataset of reviews that follows the following structure:
- id_review
- review
- original_dataset
- original_id
- service_type

### Preprocessing

Preprocessing is necessary on the European hotelreview and booking datasets because the reviews are separated into two categories: positive and negative. In our initial approach, we do not need to segment the reviews into two groups. The decision has therefore been made to combine the content of these two cells into one.
To make the pipeline easier to read, the two new dataframes thus created will be saved in CSV format.

In [ ]:
import polars as pl

df = pl.read_csv("../data/processed/dataset/data_booking.csv")

# Merge the review_positive and review_negative columns
df = df.with_columns(
    (pl.col("review_positive") + " " + pl.col("review_negative")).alias("review")
)

# Remove the original columns
df = df.drop(["review_positive", "review_negative"])

print(df.head())

# Save in CSV
df.write_csv("../data/processed/data_booking.csv")


In [ ]:
import polars as pl

df = pl.read_csv("../data/processed/dataset/data_european_hotel_reviews.csv")

# Merge the Negative_Review and Positive_Review columns
df = df.with_columns(
    (pl.col("Negative_Review") + " " + pl.col("Positive_Review")).alias("review")
)

# Remove the original columns
df = df.drop(["Negative_Review", "Positive_Review"])

print(df.head())

# Save in CSV
df.write_csv("../data/processed/data_european_hotel_reviews.csv")


## Pipeline

In [ ]:
import polars as pl

def load_dataset(path: str, review_col: str, id_col: str, dataset_name: str, service_type: str) -> pl.DataFrame:
    """
    Loads a dataset and formats it in memory in standard format:
    - review: comment text
    - original_dataset: name of the original dataset
    - original_id: original identifier
    - service_type: type of service (e.g., "accommodation", "restauration", etc.)
    """
    # Read
    df = pl.read_csv(path, encoding="latin1", ignore_errors=True)

    # Conversion to target format
    df = (
        df
        .rename({review_col: "review", id_col: "original_id"})
        .select(["review", "original_id"])
        .with_columns([
            pl.lit(dataset_name).alias("original_dataset"),
            pl.lit(service_type).alias("service_type")
        ])
        .select(["review", "original_dataset", "original_id", "service_type"])
    )

    return df


In [ ]:
# Load multiple datasets into memory
dfs = [
    load_dataset("../data/processed/dataset/data_activities_reviews.csv", "Text", "id", "data_activities_reviews", "leisure"),
    load_dataset("../data/processed/dataset/data_airline_reviews_1.csv", "ReviewBody", "id", "data_airline_reviews_1", "transportation"),
    load_dataset("../data/processed/dataset/data_airline_reviews_2.csv", "Reviews", "id", "data_airline_reviews_2", "transportation"),
    load_dataset("../data/processed/dataset/data_european_restaurant_reviews.csv", "Review", "id", "data_european_restaurant_reviews", "restauration"),
    load_dataset("../data/processed/dataset/data_hotel_reviews_1.csv", "reviews.text", "id", "data_hotel_reviews_1", "accommodation"),
    load_dataset("../data/processed/dataset/data_hotel_reviews_2.csv", "reviews.text", "id", "data_hotel_reviews_2", "accommodation"),
    load_dataset("../data/processed/dataset/data_hotel_reviews_3.csv", "reviews.text", "id", "data_hotel_reviews_3", "accommodation"),
    load_dataset("../data/processed/dataset/data_restaurant_reviews_1.csv", "Review", "id", "data_restaurant_reviews_1", "restauration"),
    load_dataset("../data/processed/dataset/data_restaurant_reviews_2.csv", "Review", "id", "data_restaurant_reviews_2", "restauration"),
    load_dataset("../data/processed/dataset/data_tripadvisor_hotel_reviews.csv", "text", "id", "data_tripadvisor_hotel_reviews", "accommodation"),
    load_dataset("../data/processed/dataset/data_twitter.csv", "Text", "id", "data_twitter", "undetermined"),
    load_dataset("../data/processed/data_booking.csv", "review", "id", "data_booking", "accommodation"),
    load_dataset("../data/processed/data_european_hotel_reviews.csv", "review", "id", "data_european_hotel_reviews", "accommodation")
]


In [ ]:
# Concatenating all DataFrames
df_all = pl.concat(dfs, rechunk=True)

# Adding an artificial id
id_column = pl.Series("id_review", range(df_all.height))  
df_all = pl.concat([id_column.to_frame(), df_all], how="horizontal")

# Manage Yelp
#df_yelp = load_dataset("../data/processed/dataset/data_yelp_reviews.csv", "text", "id", "data_yelp_reviews", "accommodation")

#df_all = pl.concat(df_yelp, rechunk=True)
# Sauvegarde du dataset global
df_all.write_csv("../data/processed/all_reviews.csv")

print("The global dataset has been created: data/processed/all_reviews.csv")
print(df_all.shape)
print(df_all.head())

In [ ]:
# Attempt

# Yelp processing
import polars as pl
df = pl.read_ndjson('../data/original/yelp_dataset/yelp_academic_dataset_review.json')
print(df.head())

df_yelp = (
    df
    .rename({"text": "review", "review_id": "original_id"}).select(["review", "original_id"])
    .with_columns([
        pl.lit("data_yelp_reviews").alias("original_dataset"),
        pl.lit("accommodation").alias("service_type")
    ])
    .select(["review", "original_dataset", "original_id", "service_type"])
)

id_column = pl.Series("id_review", range(3363667, 3363667 + df_yelp.height)) 
df_yelp = pl.concat([id_column.to_frame(), df_yelp], how="horizontal")

print(df_yelp.head())
df_yelp.write_csv("../data/processed/all_reviews_yelp.csv")



#df_all = pl.concat([df, df_yelp], rechunk=True)
#df_all.write_csv("../data/processed/all_reviews_new.csv")

### Concatenate the two CSV files in streaming mode to avoid memory overload.

In [ ]:
import polars as pl

# Read in streaming mode
df1 = pl.scan_csv("../data/processed/all_reviews.csv")
df2 = pl.scan_csv("../data/processed/all_reviews_yelp.csv")

df1 = df1.with_columns(pl.col("original_id").cast(pl.Utf8))
df2 = df2.with_columns(pl.col("original_id").cast(pl.Utf8))

# Lazy concatenation
merged = pl.concat([df1, df2])

# Writing to a new file, still streaming
merged.sink_csv("../data/processed/all_merge_reviews.csv")


## Merge only good reviews by categories

In [8]:
import polars as pl

def process_category(category_name: str, dataset_name: str, df_kw_path_template: str,
                     df_final_path_template: str, output_path_template: str):
    """
    Process reviews for a given category and dataset:
    - load the final reviews
    - join with keywords
    - rename the cleaned review column
    - remove 'original_index'
    - add 'original_dataset' and 'service_type'
    - filter by category
    - write to CSV
    """
    # Construire les chemins
    df_kw_path = df_kw_path_template.format(dataset=dataset_name)
    df_final_path = df_final_path_template.format(category=category_name, dataset=dataset_name)
    output_path = output_path_template.format(category=category_name, dataset=dataset_name)

    # Load final reviews
    df_final = pl.read_csv(df_final_path)
    
    # Load keywords
    df_kw = pl.read_csv(df_kw_path)
    
    # Join sur 'id'
    df = df_final.join(df_kw, on="id", how="left")
    
    # Rename colonne
    df = df.rename({"review_right": "review_cleaned"})
    
    # Retirer colonne 'original_index' si elle existe
    if "original_index" in df.columns:
        df = df.drop("original_index")
    
    # Ajouter colonne 'original_dataset'
    df = df.with_columns(pl.lit(dataset_name).alias("original_dataset"))
    
    # Déterminer le type de service
    dataset_lower = dataset_name.lower()
    if "hotel" in dataset_lower or "booking" in dataset_lower:
        service_type = "accommodation"
    elif "restaurant" in dataset_lower:
        service_type = "restauration"
    elif "airline" in dataset_lower:
        service_type = "transportation"
    else:
        service_type = "unknown"
    
    df = df.with_columns(pl.lit(service_type).alias("service_type"))
    
    # Filtrer par catégorie
    df = df.filter(pl.col("category") == category_name)
    
    # Écrire CSV
    df.write_csv(output_path)
    print(f"Processed category '{category_name}' for dataset '{dataset_name}' -> {output_path}")


# Paramètres généraux
categories = ["child", "pet", "handicap"]
datasets = ["data_hotel_reviews_1", "data_restaurant_reviews_1"]  # exemple
df_kw_path_template = "../data/processed/Bazarre/key_words_{dataset}.csv"
df_final_path_template = "../data/processed/final/{category}/validated_{dataset}_{category}_good.csv"
output_path_template = "processed_{dataset}_{category}.csv"

# Exécution
for dataset in datasets:
    for category in categories:
        process_category(category, dataset, df_kw_path_template, df_final_path_template, output_path_template)


Processed category 'child' for dataset 'data_hotel_reviews_1' -> processed_data_hotel_reviews_1_child.csv
Processed category 'pet' for dataset 'data_hotel_reviews_1' -> processed_data_hotel_reviews_1_pet.csv
Processed category 'handicap' for dataset 'data_hotel_reviews_1' -> processed_data_hotel_reviews_1_handicap.csv
Processed category 'child' for dataset 'data_restaurant_reviews_1' -> processed_data_restaurant_reviews_1_child.csv


SchemaError: datatypes of join keys don't match - `id`: str on left does not match `id`: i64 on right (and no other type was available to cast to)

In [10]:
import polars as pl

def process_category(category_name: str, dataset_name: str, df_kw_path_template: str,
                     df_final_path_template: str, output_path_template: str):
    """
    Process reviews for a given category and dataset safely:
    - load the final reviews (skip if empty)
    - join with keywords
    - rename the cleaned review column
    - remove 'original_index'
    - add 'original_dataset' and 'service_type'
    - filter by category
    - write to CSV
    """
    # Construire les chemins
    df_kw_path = df_kw_path_template.format(dataset=dataset_name)
    df_final_path = df_final_path_template.format(category=category_name, dataset=dataset_name)
    output_path = output_path_template.format(category=category_name, dataset=dataset_name)

    # Load final reviews
    try:
        df_final = pl.read_csv(df_final_path)
    except FileNotFoundError:
        print(f"File not found: {df_final_path}, skipping...")
        return

    # Vérifier si le CSV n'est pas vide (seulement l'en-tête)
    if df_final.is_empty():
        print(f"No reviews for category '{category_name}' in dataset '{dataset_name}', skipping...")
        return
    
    # Load keywords
    df_kw = pl.read_csv(df_kw_path)
    
    # Join sur 'id'
    df = df_final.join(df_kw, on="id", how="left")
    
    # Rename colonne
    df = df.rename({"review_right": "review_cleaned"})
    
    # Retirer colonne 'original_index' si elle existe
    if "original_index" in df.columns:
        df = df.drop("original_index")
    
    # Ajouter colonne 'original_dataset'
    df = df.with_columns(pl.lit(dataset_name).alias("original_dataset"))
    
    # Déterminer le type de service
    dataset_lower = dataset_name.lower()
    if "hotel" in dataset_lower or "booking" in dataset_lower:
        service_type = "accommodation"
    elif "restaurant" in dataset_lower:
        service_type = "restauration"
    elif "airline" in dataset_lower:
        service_type = "transportation"
    else:
        service_type = "unknown"
    
    df = df.with_columns(pl.lit(service_type).alias("service_type"))
    
    # Filtrer par catégorie
    df = df.filter(pl.col("category") == category_name)
    
    # Vérifier si le filtrage n'a pas tout supprimé
    if df.is_empty():
        print(f"No matching reviews for category '{category_name}' after filtering in dataset '{dataset_name}', skipping...")
        return
    
    # Écrire CSV
    df.write_csv(output_path)
    print(f"Processed category '{category_name}' for dataset '{dataset_name}' -> {output_path}")


# Exemple d'utilisation
categories = ["child", "pet", "handicap"]
datasets = ["data_hotel_reviews_1", "data_restaurant_reviews_1"]
df_kw_path_template = "../data/processed/Bazarre/key_words_{dataset}.csv"
df_final_path_template = "../data/processed/final/{category}/validated_{dataset}_{category}_good.csv"
output_path_template = "processed_{dataset}_{category}.csv"

for dataset in datasets:
    for category in categories:
        process_category(category, dataset, df_kw_path_template, df_final_path_template, output_path_template)


Processed category 'child' for dataset 'data_hotel_reviews_1' -> processed_data_hotel_reviews_1_child.csv
Processed category 'pet' for dataset 'data_hotel_reviews_1' -> processed_data_hotel_reviews_1_pet.csv
Processed category 'handicap' for dataset 'data_hotel_reviews_1' -> processed_data_hotel_reviews_1_handicap.csv
Processed category 'child' for dataset 'data_restaurant_reviews_1' -> processed_data_restaurant_reviews_1_child.csv
No reviews for category 'pet' in dataset 'data_restaurant_reviews_1', skipping...
File not found: ../data/processed/final/handicap/validated_data_restaurant_reviews_1_handicap_good.csv, skipping...


In [11]:
import polars as pl
import glob
import os

categories = ["child", "pet", "handicap"]

for category in categories:
    # Récupérer tous les CSV pour cette catégorie
    pattern = f"processed_*_{category}.csv"
    csv_files = glob.glob(pattern)
    
    if not csv_files:
        print(f"No processed CSVs found for category '{category}', skipping merge.")
        continue
    
    # Lire et concaténer tous les CSV
    dfs = []
    for f in csv_files:
        df = pl.read_csv(f)
        dfs.append(df)
    
    merged_df = pl.concat(dfs)
    
    # Écrire le CSV final pour la catégorie
    final_output = f"{category}_all.csv"
    merged_df.write_csv(final_output)
    print(f"Merged {len(csv_files)} files into {final_output}")
    
    # Supprimer les CSV intermédiaires
    for f in csv_files:
        os.remove(f)
    print(f"Deleted intermediate files for category '{category}'")


Merged 2 files into child_all.csv
Deleted intermediate files for category 'child'
Merged 1 files into pet_all.csv
Deleted intermediate files for category 'pet'
Merged 1 files into handicap_all.csv
Deleted intermediate files for category 'handicap'
